# <a id='toc1_'></a>[CubeSharp Examples](#toc0_)

Task-oriented, progressively complex usage scenarios. See 01-Tutorial.ipynb for the conceptual walkthrough.

**Table of Contents**<a id='toc0_'></a>    
- [CubeSharp Examples](#toc1_)    
  - [Prerequisites](#toc1_1_)    
    - [Helper Functions](#toc1_1_1_)    
      - [Visualization helpers](#toc1_1_1_1_)    
      - [Cube -> Table helper](#toc1_1_1_2_)    
  - [Sample Data](#toc1_2_)    
  - [1D Report: Total quantity per customer](#toc1_3_)    
  - [2D Report: Quantity per year per customer](#toc1_4_)    
  - [2D Report: Quantity per tag per year (multi-selection)](#toc1_5_)    
  - [3D Report: EmployeeId x CustomerId x Year](#toc1_6_)    
  - [Zero / simple aggregation examples](#toc1_7_)    
  - [Modulus dimension examples](#toc1_8_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Prerequisites](#toc0_)
This cell installs required NuGet packages and sets up global using directives.

In [ ]:
// Dependencies
#r "nuget: CubeSharp, 99.0.0.3-alpha"

using CubeSharp;


### <a id='toc1_1_1_'></a>[Helper Functions](#toc0_)

#### <a id='toc1_1_1_1_'></a>[Visualization helpers](#toc0_)

In [ ]:
#r "nuget: Microsoft.Data.Analysis, 0.21.0"
#nullable enable

using System;
using System.Collections.Generic;
using System.Linq;
using System.Reflection;
using Microsoft.Data.Analysis;

public static void Display(this object? obj) => display(obj);
public static void DisplayAsTable(this IEnumerable<IDictionary<string, object?>> rows) => display(rows.ToDataFrame());
public static IDictionary<string, object?> ObjectToDictionary(this object obj) =>
    obj.GetType().GetProperties(BindingFlags.Public | BindingFlags.Instance)
        .ToDictionary(prop => prop.Name, prop => prop.GetValue(obj, null));
private static DataFrame ToDataFrame(this IEnumerable<IDictionary<string, object?>> source)
{
    var props = source.FirstOrDefault()?.Keys ?? [];
    var columns = props.Select(p => new StringDataFrameColumn(p)).ToList();
    var result = new DataFrame(columns);
    foreach (var item in source)
    {
        result.Append(props.ToDictionary(p => p, p => GetDisplayedValue(item[p])), true);
    }
    return result;
    static object? GetDisplayedValue(object? value) => value switch
    {
        IEnumerable<object> values => $"[{values.JoinStrings(", " )}]",
        _ => value
    };
}
private static string JoinStrings(this IEnumerable<object?> strings, string separator) => string.Join(separator, strings);


#### <a id='toc1_1_1_2_'></a>[Cube -> Table helper](#toc0_)

In [ ]:
#nullable enable
public static IEnumerable<IDictionary<string, object?>> ToTable<TIndex, T>(
    this CubeResult<TIndex, T> cubeResult, Index[] rowDimensionNumbers, Index[] columnDimensionNumbers) where TIndex : notnull
{
    if (rowDimensionNumbers.Concat(columnDimensionNumbers).GroupBy(x => x).Any(g => g.Count() > 1))
    {
        throw new InvalidOperationException("Duplicated indexes are not allowed");
    }
    var shiftedColumnIndexes =
        ShiftIndexes(columnDimensionNumbers, rowDimensionNumbers, cubeResult.FreeDimensionCount);
    return cubeResult
        .BreakdownByDimensions(rowDimensionNumbers)
        .Select(row => row
            .GetBoundDimensionsAndIndexes()
            .Select(item => KeyValuePair.Create(
                item.dimension.Title ?? string.Empty,
                (object?)item.dimension[item.index].Title ?? item.index))
            .Concat(row
                .BreakdownByDimensions(shiftedColumnIndexes)
                .Select(column => KeyValuePair.Create(
                    column
                        .GetBoundDimensionsAndIndexes()[^columnDimensionNumbers.Length ..]
                        .Select(item => item.dimension[item.index].Title ?? item.index?.ToString())
                        .DefaultIfEmpty("value")
                        .JoinStrings("."),
                    (object?)column.GetValue())))
            .ToDictionary(kvp => kvp.Key, kvp => kvp.Value));
}
private static Index[] ShiftIndexes(Index[] indexes, Index[] precedingIndexes, int length) =>
    indexes
        .Select(index => index.GetOffset(length))
        .Select(index =>
            index - precedingIndexes.Count(i => i.GetOffset(length) < index))
        .Select(dimensionNumber => (Index)dimensionNumber)
        .ToArray();



## <a id='toc1_2_'></a>[Sample Data](#toc0_)

In [ ]:
var orders = new [] {
    new { OrderDate = new DateTime(2007, 08, 02), Product = "X", EmployeeId = 3, CustomerId = "A", Quantity = 10m, Tags = new [] { "Discount", "Retail" } },
    new { OrderDate = new DateTime(2007, 12, 24), Product = "X", EmployeeId = 3, CustomerId = "A", Quantity = 12m, Tags = new [] { "Retail", "BestSeller" } },
    new { OrderDate = new DateTime(2007, 12, 24), Product = "Y", EmployeeId = 1, CustomerId = "B", Quantity = 20m, Tags = new [] { "Discount", "Retail", "New", "Season", "BestSeller" } },
    new { OrderDate = new DateTime(2008, 01, 09), Product = "Z", EmployeeId = 2, CustomerId = "A", Quantity = 40m, Tags = new [] { "BestSeller" } },
    new { OrderDate = new DateTime(2008, 01, 18), Product = "Z", EmployeeId = 1, CustomerId = "C", Quantity = 14m, Tags = new [] { "Discount", "New", "BestSeller" } },
    new { OrderDate = new DateTime(2008, 02, 12), Product = "Z", EmployeeId = 2, CustomerId = "B", Quantity = 12m, Tags = new [] { "Retail" } },
    new { OrderDate = new DateTime(2009, 02, 12), Product = "X", EmployeeId = 3, CustomerId = "A", Quantity = 10m, Tags = new string[] {} },
    new { OrderDate = new DateTime(2009, 02, 16), Product = "X", EmployeeId = 1, CustomerId = "C", Quantity = 20m, Tags = new [] { "New" } },
    new { OrderDate = new DateTime(2009, 04, 18), Product = "Z", EmployeeId = 2, CustomerId = "B", Quantity = 15m, Tags = new [] { "Discount", "BestSeller" } },
    new { OrderDate = new DateTime(2007, 04, 18), Product = "X", EmployeeId = 3, CustomerId = "C", Quantity = 22m, Tags = new [] { "Discount", "Retail", "New", "Season" } },
    new { OrderDate = new DateTime(2009, 09, 07), Product = "Y", EmployeeId = 3, CustomerId = "D", Quantity = 30m, Tags = new [] { "Retail", "New", "Season", "BestSeller" } },
    new { OrderDate = new DateTime(2009, 09, 07), Product = (string)null, EmployeeId = 3, CustomerId = "D", Quantity = 30m, Tags = new [] { "Discount", "New", "BestSeller" } },
};
orders.Select(order => order.ObjectToDictionary()).DisplayAsTable();


## <a id='toc1_3_'></a>[1D Report: Total quantity per customer](#toc0_)

In [ ]:
var cube1d = orders.BuildCube(
    AggregationDefinition.CreateForCollection(orders, o => o.Quantity, (a,b) => a + b, 0m),
    DimensionDefinition.CreateForCollection(orders, o => o.CustomerId, null,
        IndexDefinition.Create("A"), IndexDefinition.Create("B"), IndexDefinition.Create("C"), IndexDefinition.Create("D")));
new { A = cube1d.GetValue("A"), B = cube1d.GetValue("B"), C = cube1d.GetValue("C"), D = cube1d.GetValue("D") }.Display();


## <a id='toc1_4_'></a>[2D Report: Quantity per year per customer](#toc0_)

In [ ]:
var cube2d = orders.BuildCube(
    AggregationDefinition.CreateForCollection(orders, o => o.Quantity, (a,b)=>a+b, 0m),
    DimensionDefinition.CreateForCollection(orders, o => o.CustomerId, null,
        IndexDefinition.Create("A"), IndexDefinition.Create("B"), IndexDefinition.Create("C"), IndexDefinition.Create("D"))
        .WithTrailingDefaultIndex("Total"),
    DimensionDefinition.CreateForCollection(orders, o => o.OrderDate.Year.ToString(), null,
        IndexDefinition.Create("2007"), IndexDefinition.Create("2008"), IndexDefinition.Create("2009")));
var tableCustomersYears = cube2d.BreakdownByDimensions(0)
    .Select(row => new {
        Customer = row.GetBoundIndexDefinition(0).Title ?? row.GetBoundIndexDefinition(0).Value,
        Y2007 = row.GetValue("2007"),
        Y2008 = row.GetValue("2008"),
        Y2009 = row.GetValue("2009"),
        Total = row.GetValue((string)default!)
    })
    .Select(order => order.ObjectToDictionary())
    .ToList();
tableCustomersYears.DisplayAsTable();


## <a id='toc1_5_'></a>[2D Report: Quantity per tag per year (multi-selection)](#toc0_)

In [ ]:
var cubeTagYear = orders.BuildCube(
    AggregationDefinition.CreateForCollection(orders, o => o.Quantity, (a,b)=>a+b, 0m),
    DimensionDefinition.CreateForCollection(orders, o => o.OrderDate.Year.ToString(), "Year",
        IndexDefinition.Create("2007"), IndexDefinition.Create("2008"), IndexDefinition.Create("2009"))
        .WithTrailingDefaultIndex("Total"),
    DimensionDefinition.CreateForCollectionWithMultiSelector(orders, o => o.Tags, null,
        IndexDefinition.Create("Discount"), IndexDefinition.Create("Retail"), IndexDefinition.Create("New"), IndexDefinition.Create("Season"), IndexDefinition.Create("BestSeller")));
var tagYear = cubeTagYear.BreakdownByDimensions(0)
    .Select(row => new {
        Year = row.GetBoundIndexDefinition(0).Title ?? row.GetBoundIndexDefinition(0).Value,
        Discount = row.GetValue("Discount"),
        Retail = row.GetValue("Retail"),
        New = row.GetValue("New"),
        Season = row.GetValue("Season"),
        BestSeller = row.GetValue("BestSeller")
    })
    .Select(order => order.ObjectToDictionary())
    .ToList();
tagYear.DisplayAsTable();


## <a id='toc1_6_'></a>[3D Report: EmployeeId x CustomerId x Year](#toc0_)

In [ ]:
var cube3d = orders.BuildCube(
    AggregationDefinition.CreateForCollection(orders, o => o.Quantity, (a,b)=>a+b, 0m),
    DimensionDefinition.CreateForCollection(orders, o => o.EmployeeId.ToString(), "EmployeeId", IndexDefinition.Create("1"), IndexDefinition.Create("2"), IndexDefinition.Create("3")).WithTrailingDefaultIndex("Total"),
    DimensionDefinition.CreateForCollection(orders, o => o.CustomerId, "CustomerId", IndexDefinition.Create("A"), IndexDefinition.Create("B"), IndexDefinition.Create("C"), IndexDefinition.Create("D")).WithTrailingDefaultIndex("Total"),
    DimensionDefinition.CreateForCollection(orders, o => o.OrderDate.Year.ToString(), "Year", IndexDefinition.Create("2007"), IndexDefinition.Create("2008"), IndexDefinition.Create("2009")).WithTrailingDefaultIndex("Total"));
var table3d = cube3d.ToTable([0, 1], [2]);
table3d.DisplayTable();


## <a id='toc1_7_'></a>[Zero / simple aggregation examples](#toc0_)
Hello world

In [ ]:
Array.Empty<string>()
    .BuildCube<string, string, string>(AggregationDefinition.Create((string x) => x, (a, b) => default!, "Hello world"))
    .GetValue()
    .Display();


Sum of numbers from 1 to 10

In [ ]:
Enumerable.Range(1,10)
    .BuildCube<int, int, int>(AggregationDefinition.Create((int i) => i, (a, b) => a + b, 0))
    .GetValue()
    .Display();


Array of numbers from 1 to 10

In [ ]:
Enumerable.Range(0, 10)
    .BuildCube<int, int, int[]>(AggregationDefinition.Create((int i) => new[] { i }, (a,b) => a.Concat(b).ToArray(), []))
    .GetValue()
    .Display();


## <a id='toc1_8_'></a>[Modulus dimension examples](#toc0_)

In [ ]:
var mod3Cube = Enumerable.Range(0,10).BuildCube(
    AggregationDefinition.Create((int i) => new[] { i }, (a, b) => a.Concat(b).ToArray(), Array.Empty<int>()),
    DimensionDefinition.Create((int i) => (int?)(i % 3), "Mod 3", Enumerable.Range(0, 3).Select(i => IndexDefinition.Create((int?)i)).ToArray()));
mod3Cube.GetValue().Display(); // x % 3 = *
mod3Cube.GetValue(0).Display(); // x % 3 = 0
mod3Cube.GetValue(1).Display(); // x % 3 = 1
mod3Cube.GetValue(2).Display(); // x % 3 = 2


In [ ]:
DimensionDefinition<int, int?> CreateModulusDimension(int m) => DimensionDefinition.Create((int i)=> (int?)(i % m), $"Mod {m}", Enumerable.Range(0,m).Select(i => IndexDefinition.Create((int?)i)).ToArray());
var mod3and5Cube = Enumerable.Range(0,20).BuildCube(
    AggregationDefinition.Create((int i)=> new[] { i }, (a, b) => a.Concat(b).ToArray(), Array.Empty<int>()),
    CreateModulusDimension(3),
    CreateModulusDimension(5));
mod3and5Cube.GetValue(0).Display();       // x % 3 = 0
mod3and5Cube.GetValue(null, 0).Display(); // x % 5 = 0
mod3and5Cube.GetValue(0, 0).Display();    // x % 3 = 0 and x % 5 = 0
mod3and5Cube.ToTable([0, 1], []).DisplayTable();
